**DTS - Complete Streets** *Right-of-Way Widths for Planned Street Improvements*



# 05 - Modal Widths

**Author:** rmangan

____

**Purpose:**

Utilize all previously compiled attributes in *modal composite 05* in conjunction with standardized width parameters for elements in the street cross section to calcualte Unconstrained Modal Width, Area, and other metrics.

**This script performs the following functions:**

1. Pre-Processing & Cleanup
   1. Drop unneeded fields, add required fields etc
   2. Convert 1986 ROW table values to numbers.
2. Modal Width Development - Develop individual modal widths per modal type.
3. Calculate Unconstrained Metrics - Summarize modal widths into Total Unconstrained width, area, and other values.
4. (optional utility functions) - reset all values to 0 for reprocessing.


**Global Assumptions and Notes**

1. Modal Areas are calcualted as Street Segment Length x Modal Width
2. All widths values are in feet. Area values are in square feet.


**Non-Standard Python Modules utilized:**
1. arcpy 2.7 - Geoprocessing
2. pandas 1.1 - Results confirmation





### Setup

In [1]:
# import modules
import arcpy
import os
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS

In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\modal\Modal_Composite4.gdb"
arcpy.env.overwriteOutput = True

In [2]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

scratch_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"

output_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_output.gdb"


# Input Datasets

HPMS = os.path.join(input_gdb_path,"HPMS_RCL_join")

HERE = os.path.join(input_gdb_path,"RCLs_with_NAVOIDs_35buff_10clip")

HERE_orig = os.path.join(input_gdb_path, "hw_linksSPEED")

medians = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\median_classification.gdb\modal_comp_05_median_classification"

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"

modal_composite_url = r"https://services6.arcgis.com/2cZSk3EXXiOHcbOl/ArcGIS/rest/services/Modal_Composite_5_1_publish_gdb/FeatureServer/0"

modal_composite_url_copy = r"https://services6.arcgis.com/2cZSk3EXXiOHcbOl/ArcGIS/rest/services/Modal_Composite_5_1_publish_gdb/FeatureServer/0"

zoning = r"\\dc1vs01\GISProj\B\Base\HI\Honolulu_City_County\zoning\CCH_zoning.gdb\zoning_LUO_20210505"

# 1.  Pre-Processing & Clean up

### Drop Unecessary Fields

In [ ]:
#drop unecessary fields (all fields are avaialble in modal_composite_full dataset)

#get all fields and add to list
# field_list = []

# fields = arcpy.ListFields(modal_composite)

# for field in fields:
#     field_list.append(field.name)
    
field_list = [i.name for i in arcpy.ListFields(modal_composite)]

# for i in field_list:
#     print("\"{}\",".format(i))

#fields to keep

keep_fields = [
    "OBJECTID",
    "Shape",
    "SEGMENTID",
    "FULLNAME",
    "NAME",
    "PROPERNAME",
    "TYPE",
    "POSTDIR",
    "CLASS",
    "STREET_CLA",
    "ZIPCODER",
    "ZIPCODEL",
    "TOWNL",
    "TOWNR",
    "NEIGHBORL",
    "NEIGHBORR",
    "ONEWAY",
    "OWNER",
    "NBRightPed",
    "SBLeftPedZ",
    "NBRightP_1",
    "SBLeftPe_1",
    "NBRightP_2",
    "SBLeftPe_2",
    "NBRightP_3",
    "SBLeftPe_3",
    "NBRightP_4",
    "SBLeftPe_4",
    "NBRightP_5",
    "SBLeftPedB",
    "NBRightP_6",
    "SBLeftPe_5",
    "NBRightP_7",
    "SBLeftPe_6",
    "NBRightP_8",
    "SBLeftPe_7",
    "NBRightVis",
    "SBLeftVisu",
    "MidBlkXwkU",
    "MidBlkXwkC",
    "MidBlkXw_1",
    "TraffCalmi",
    "PPN_Final",
    "HPI",
    "PpnNotCity",
    "FID_Ped_Improve_temp",
    "Ped_Improve_ProjectID",
    "Ped_Immprove_Extents",
    "FID_Ped_Add_temp",
    "Ped_Add_Project_ID",
    "Fac_Name_BE",
    "Fac_Desc_BE",
    "Fac_Type_BE",
    "Fac_Name_BP",
    "Fac_Desc_BP",
    "Fac_Name_BR",
    "Fac_Desc_BR",
    "Fac_Type_BR",
    "TCAD_OID_2020",
    "AB_SPEED",
    "BA_SPEED",
    "AB_CAPACIT",
    "BA_CAPACIT",
    "AB_LANE",
    "BA_LANE",
    "AB_CNT",
    "BA_CNT",
    "AB_FACTYPE",
    "BA_FACTYPE",
    "AB_FNCLASS",
    "BA_FNCLASS",
    "TOT_FLOW_D",
    "max_row_2020",
    "min_row_2020",
    "mean_row_2020",
    "f_class",
    "fedaid_id",
    "ln_exist",
    "ln_prop",
    "row_exist",
    "row_prop",
    "TCAD_OID_2040",
    "ID",
    "AB_VOC_MAX",
    "BA_VOC_MAX",
    "FACTYPE",
    "ATYPE",
    "TPN_OID",
    "FULLNAME_1",
    "Tier",
    "NewTier",
    "BE_OID_tmp",
    "BP_OID_tmp",
    "BR_OID_tmp",
    "PED_ADD_OID_tmp",
    "PED_IMPR_OID_tmp",
    "Shape_Length",
    "ln_exist_num",
    "ln_prop_num",
    "row_exist_num",
    "row_prop_num",
    "modal_width_ped_exist",
    "modal_width_ped_prop",
    "modal_width_bike_exist",
    "modal_width_bike_prop",
    "modal_width_auto_exist",
    "modal_width_auto_prop",
    "modal_width_bus_exist",
    "modal_width_bus_prop",
    "modal_width_park_exist",
    "modal_width_park_prop",
    "modal_width_medians",
    "modal_width_max",
    "Fac_Type_BP",
    "lanes_assumed",
    "VOC_max",
    "median_type",
    "median_width",
    "progression",
    "CS_example_num",
    "CS_example_type",
    "CS_type",
    "length_urban_percent",
    "length_urban",
    "length_ag",
    "length_ag_percent",
    "length_ft",
    "mean_row_area",
    "modal_area_max",
    "modal_area_diff",
    "route_id",
    "aadt",
    "funsystem",
    "funsysdesc",
    "factype_1",
    "factypedes",
    "thrulanes",
    "SPEED",
    "SPD_LIMIT",
    "MIN_SPD_LIMIT",
    "MAX_SPD_LIMIT",
    "LandValue",
    "modal_width_ped_const",
    "modal_width_bike_const",
    "modal_width_auto_const",
    "modal_width_bus_const",
    "modal_width_park_const",
    "modal_width_medians_const",
    "modal_width_const",
    "modal_area_const",
    "modal_area_diff_const",
    "bike_parallel_route",
    "lane_count_rev",
    "cs_type_revise",
    "modal_width_auto_rule",
    "modal_width_ped_furn",
    "row_estimate_note",
    "modal_area_auto_E",
    "modal_area_auto_U",
    "modal_area_auto_C",
    "modal_area_ped_E",
    "modal_area_ped_U",
    "modal_area_ped_C",
    "modal_area_bike_E",
    "modal_area_bike_U",
    "modal_area_bike_C"
]

In [ ]:
#get all fields and check against keep list to create drop list
list_fields = arcpy.ListFields(modal_composite)
fields = [i.name for i in list_fields]

drop_list = []
for i in fields:
    if i in keep_fields:
        print("{} field - keeping".format(i))
    else:
        print("{} field - dropping".format(i))
        drop_list.append(i)

#drop fields
print("dropping fields..")
arcpy.DeleteField_management(modal_composite, drop_list)

print("done")

### Add Required Fields

In [ ]:
#add field for analysis if not already present

#store field, field alias in dict

#widths = SHORT
#areas = FLOAT
#misc = TEXT

new_fields = {"row_estimate_note":      "ROW Estimation Note", #TEXT
              "cs_type_revise":         "Complete Street Type - Re-Evaluate",
              
              "lanes_assumed":     "Lanes (assumed)",
              "lane_count_exist":  "Transit Lanes - Existing",
              "lane_count_prop":   "Transit Lanes - Proposed (Unconstrained)",
              "lane_count_rev":    "Transit Lanes - Proposed (Constrained)",
              "lane_miles_exist":  "Transit Lane Miles - Existing",
              "lane_miles_prop":   "Transit Lane Miles - Proposed (Unconstrained)",
              "lane_miles_const":  "Transit Lane Miles - Proposed (Constrained)",
              "lane_count_note":   "Transit Lanes - Note",    #TEXT
              
              "modal_width_ped_exist":  "Modal Width - Ped (Existing)",
              "modal_width_ped_prop":   "Modal Width - Ped (Unconstrained)",
              "modal_width_ped_const":  "Modal Width - Ped (Constrained)",
              "modal_area_ped_exist":   "Modal Area - Ped (Existing)",
              "modal_area_ped_prop":    "Modal Area - Ped (Unconstrained)",
              "modal_area_ped_const":   "Modal Area - Ped (Constrained)",
              
              "modal_width_bike_exist": "Modal Width - Bike (Existing)",
              "modal_width_bike_prop":  "Modal Width - Bike (Unconstrained)",
              "modal_width_bike_const": "Modal Width - Bike (Constrained)",
              "modal_area_bike_exist":  "Modal Area - Bike (Existing)",
              "modal_area_bike_prop":   "Modal Area - Bike (Unconstrained)",  
              "modal_area_bike_const":  "Modal Area - Bike (Constrained)",  
              "bike_parallel_route":    "Bike - Evaluate Parallel Routes",        
              
              "modal_width_auto_exist": "Modal Width - Auto (Existing)",
              "modal_width_auto_prop":  "Modal Width - Auto (Unconstrained)",
              "modal_width_auto_const": "Modal Width - Auto (Constrained)",
              "modal_area_auto_exist":  "Modal Area - Auto (Existing)",
              "modal_area_auto_prop":   "Modal Area - Auto (Unconstrained)",
              "modal_area_auto_const":  "Modal Area - Auto (Constrained)",
              "modal_width_auto_rule":  "Modal Width - Auto (Logic QC)",   #TEXT
              
              "modal_width_bus_exist":  "Modal Width - Bus (Existing)",
              "modal_width_bus_prop":   "Modal Width - Bus (Proposed)",
              "modal_width_bus_const":  "Modal Width - Bus (Constrained)",
              "modal_area_bus_exist":  "Modal Area - Bus (Existing)",
              "modal_area_bus_prop":   "Modal Area - Bus (Unconstrained)",
              "modal_area_bus_const":  "Modal Area - Bus (Constrained)",      
              
              "modal_width_park_exist": "Modal Width - Parking (Existing)",
              "modal_width_park_prop":  "Modal Width - Parking (Proposed)",
              "modal_width_park_const": "Modal Width - Parking (Constrained)",
              "modal_area_park_exist":  "Modal Area - Parking (Existing)",
              "modal_area_park_prop":   "Modal Area - Parking (Unconstrained)",
              "modal_area_park_const":  "Modal Area - Parking (Constrained)",           
              
              "modal_width_medians_exist":  "Modal Width - Medians (Existing)",
              "modal_width_medians_prop":  "Modal Width - Medians (Proposed)",
              "modal_width_medians_const":  "Modal Width - Medians (Constrained)",
              "modal_area_medians_exist":  "Modal Area - Medians (Existing)",
              "modal_area_medians_prop":   "Modal Area - Medians (Unconstrained)",
              "modal_area_medians_const":  "Modal Area - Medians (Constrained)",
                 
              "length_ft":              "Length (ft.)",
              "mean_row_area":          "ROW Area Estimate (sq. ft.)",
              
              "modal_width_exist":      "Width - Existing",
              "modal_width_max":        "Width - Unconstrained",
              "modal_width_max_diff":   "Width - Unconstrained Shortage",
              "modal_width_max_desc":   "Unconstrained Description", #TEXT
              "modal_width_const":      "Width - Constrained",
              "modal_width_const_diff": "Width - Constrained Shortage",
              "modal_width_const_desc": "Constrained Description",   #TEXT
  
              "modal_area_max":             "Area - Unconstrained (sq. ft.)",
              "modal_area_max_diff":        "Area - Unconstrained Shortage (sq. ft.)",
              "modal_area_const":           "Area - Constrained (sq. ft.)",
              "modal_area_const_diff":      "Area - Unconstrained Shortage (sq. ft.)"}

#loop through dict and add fields
    
#create list of field names
fields = [i.name for i in arcpy.ListFields(modal_composite)]

for key, value in new_fields.items():
    if key in fields:
        print("Field {} already exists".format(key))
        print("Updating alias to: {}".format(value))
        arcpy.AlterField_management(in_table = modal_composite,field = key, new_field_alias = value)
           
    else:
        if key in ["lane_count_note",
                   "modal_width_auto_rule",
                   "row_estimate_note",
                   "modal_width_max_desc",
                   "modal_width_const_desc"]:
            print("Adding field {0}".format(key))
            arcpy.AddField_management(modal_composite,field_name=key,field_type="TEXT", field_alias = value)
            
        elif key in ["lane_miles_exist",
                    "lane_miles_prop",
                    "lane_miles_const",  
                    "modal_area_ped_exist",
                    "modal_area_ped_prop",
                    "modal_area_ped_const",           
                    "modal_area_bike_exist",
                    "modal_area_bike_prop",  
                    "modal_area_bike_const",  
                    "modal_area_auto_exist",
                    "modal_area_auto_prop",
                    "modal_area_auto_const",      
                    "modal_area_bus_exist",
                    "modal_area_bus_prop",
                    "modal_area_bus_const",                  
                    "modal_area_park_exist",
                    "modal_area_park_prop",
                    "modal_area_park_const",           
                    "modal_area_medians_exist",
                    "modal_area_medians_prop",
                    "modal_area_medians_const",               
                    "length_ft",
                    "mean_row_area",
                    "modal_area_max",
                    "modal_area_max_diff",
                    "modal_area_const",
                    "modal_area_const_diff"]:
            
            
            print("Adding field {0}".format(key))
            arcpy.AddField_management(modal_composite,field_name=key,field_type="FLOAT", field_alias = value) 
            
        else:
            print("Adding field {0}".format(key))
            arcpy.AddField_management(modal_composite,field_name=key,field_type="SHORT", field_alias = value)
                       
print("done")

### Parse 1986 ROW Table values to Numbers

In [ ]:
#create num fields from 1986 text fields & parse value to new fields handle text entry via hardcoded mapping

#copy input dataset while testing functions
print("copying modal input for processing...")
modal_processing_temp = arcpy.CopyFeatures_management(modal_composite_03,"modal_processing_temp")
print("done")

#store field, field alias in dict
new_fields = {"ln_exist_num":"1986 Existing Lanes (num)",
              "ln_prop_num": "1986 Proposed Lanes (num)",
              "row_exist_num":"1986 Existing ROW (num)",
              "row_prop_num":"1986 Proposed ROW (num)"}

#loop through dict and add fields
for key,value in new_fields.items():
    print("Adding field {0}".format(key))
    arcpy.AddField_management(modal_processing_temp,field_name=key,field_type="SHORT", field_alias = value)
          
print("Field additions complete")

In [ ]:
#field to expose to update cursor
fields = ["ln_exist","ln_prop","row_exist","row_prop","ln_exist_num","ln_prop_num", "row_exist_num","row_prop_num"]

#update ln_exist_num w/ update cursor
print("parse ln_exist to int")
with arcpy.da.UpdateCursor(modal_processing_temp,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[0] == '4+ bus':
                row[4] = 4
            elif row[0] == '3+ bus':
                row[4] = 3
            else:
                row[4]= int(row[0])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)            
print("ln_exist parsed\n")
            
#update ln_prop_num w/ update cursor
print("parse ln_prop to int")
with arcpy.da.UpdateCursor(modal_processing_temp,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[1] == '5+ bus':
                row[5] = 5
            else:
                row[5]= int(row[1])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("ln_prop parsed\n")
            
#update row_exist_num w/ update cursor
print("parse row_exist to num")
with arcpy.da.UpdateCursor(modal_processing_temp,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            if row[2] == 'Var. to 80':
                row[6] = 80
            elif row[2] == 'Var. 60-70':
                row[6] = 65
            elif row[2] == 'Var. 36-50':
                row[6] == 61
            elif row[2] == 'Var. 14-26':
                row[6] = 20
            elif row[2] == '86/100':
                row[6] = 93
            elif row[2] == '60-90':
                row[6] = 75
            elif row[2] == '60-76':
                row[6] = 68
            elif row[2] == '60-70':
                row[6] = 65
            elif row[2] == '60-64':
                row[6] = 62
            elif row[2] == '50-80':
                row[6] = 65
            elif row[2] == '50-60':
                row[6] = 55
            elif row[2] == '44-56':
                row[6] = 50
            elif row[2] == '40/50':
                row[6] = 45
            elif row[2] == '40-56':
                row[6] = 48
            elif row[2] == '30-50':
                row[6] = 40
            elif row[2] == '30-40':
                row[6] = 35
            elif row[2] == '25-50':
                row[6] = 38
            elif row[2] == '20-40':
                row[6] = 30
            elif row[2] == '25-50':
                row[6] = 38
            elif row[2] == '120-140':
                row[6] = 130
            elif row[2] == '113.5':
                row[6] = 114
            elif row[2] == '100-110':
                row[6] = 105
            elif row[2] == '0':
                row[6] = None
            else:
                row[6]= int(row[2])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("row_exist parsed\n")            
                  
#update row_prop_num w/ update cursor
print("parse row_prop to int")
with arcpy.da.UpdateCursor(modal_processing_temp,fields,'NOT "ln_exist" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:          
            if row[3] == 'Var. to 80':
                row[7] = 80
            elif row[3] == 'Var. 14-26':
                row[7] = 20
            elif row[3] == '86/100':
                row[7] = 93
            elif row[3] == '60-90':
                row[7] = 75
            elif row[3] == '60-76':
                row[7] = 68
            elif row[3] == '60-70':
                row[7] = 65
            elif row[3] == '60-64':
                row[7] = 62
            elif row[3] == '56-80':
                row[7] = 68
            elif row[3] == '50-60':
                row[7] = 55
            elif row[3] == '40/50':
                row[7] = 45
            elif row[3] == '30-40':
                row[7] = 35
            elif row[3] == '25-50':
                row[7] = 38
            elif row[3] == '120-140':
                row[7] = 130
            elif row[3] == '113.5':
                row[7] = 114
            elif row[3] == '100-110':
                row[7] = 105
            else:
                row[7] = int(row[3])
            cursor.updateRow(row)
        except ValueError as error:
            print(error)
print("row_prop parsed\n")

### Join misc data (HPMS, medians, etc...)

In [ ]:
#join fields from median classification
print("Joining Median fields...")
join_target = modal_composite
join_target_field = "SEGMENTID"
join_table = medians
join_table_field = "SEGMENTID"
join_fields = ["median_type","median_width","progression"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join Fields complete.")
    
print("Done")


In [ ]:
#Join fields from HMPS
print("Joining HPMS fields...")
join_target = modal_composite
join_target_field = "SEGMENTID"
join_table = HPMS
join_table_field = "SEGMENTID"
join_fields = ["aadt","route_id","funsystem", "funsysdesc","factype","factypedes","thrulanes"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join HPMS Fields complete.")

#Join speed limit fields from HERE for QC comparison
print("Joining HERE fields...")
join_target = modal_composite
join_target_field = "SEGMENTID"
join_table = HERE
join_table_field = "SEGMENTID"
join_fields = ["SPEED","SPD_LIMIT","MIN_SPD_LIMIT", "MAX_SPD_LIMIT"]

arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
print("Join HERE Fields complete.")
    
print("Done")

In [ ]:
#update field alias for HPMS and HERE fields

update_alias = {
    "aadt":"HPMS - Annual Average Daily Traffic, Two Directions",
    "route_id":"HPMS - Route Number",
    "funsystem":"HPMS - Functional System Code",
    "funsysdesc":"HPMS - Functional System Code Description",
    "factype":"HPMS - Facility Type Code",
    "factypedes":"HPMS - Facility Type Code Description",
    "thrulanes":"HPMS - Number of Through Lanes, Two Directions",
    "SPD_LIMIT":"Speed Limit (HERE)",
    "MIN_SPD_LIMIT":"Speed Limit (HERE - low)",
    "MAX_SPD_LIMIT":"Speed Limit (HERE - high)"
}

#rename bike proposed field
for key, value in update_alias.items():
    new_field_name = str(key)+"_BP"
    print("assigning alias {0} to field {1}".format(value,key))
    arcpy.AlterField_management(modal_composite_05_2,key,new_field_alias=value)
    
print("alias assignment done")

In [ ]:
#join lane values from HERE for QC comparison

#dissovle HERE on LinkID, summing Lane Values together
#Create near table between HERE and RCL with 3 closest matches, 20' search distance. 
#Join Street Name & Lane Summary from HERE. Join Street Name & Segment ID from RCL to near table.
#Discard records of mismatched street names. (here was close to an RCL of another street)
#Summary Stats on table, using SegmentID as case field, get Median Lane value of remaining records
#add median lane value to modal composite




#on HERE, dissolve Lanes(sum) (linkID is dissolve field)
print("dissolving Lanes on LinkID...")
here_dissolve = os.path.join(scratch_gdb_path, "here_dissolve")
arcpy.management.Dissolve(HERE_orig, here_dissolve, dissolve_field = "LINK_ID", statistics_fields = [["LANES","SUM"]])

#generate near table between dissolve output & RCL, find 3 closest features, search radius 20'
print("generating near table...")
near = os.path.join(scratch_gdb_path, "near")
arcpy.analysis.GenerateNearTable(here_dissolve, modal_composite,
                                        near, search_radius = 20,
                                        closest = False, closest_count = 3)

#on near table, join field from dissolve output IN_FID = Dissolve OID, add LinkID & Lane Sum
print("joining LinkID & Lane Summary from dissolve output...")
join_target = near
join_target_field = "IN_FID"
join_table = here_dissolve
join_table_field = "OBJECTID"
join_fields = ["LINK_ID","SUM_LANES"]
arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)

#on near table, join field from HERE LinkID = LinkID, add ST_NAME
print("joining ST_NAME from HERE...")
join_target = near
join_target_field = "LINK_ID"
join_table = HERE_orig
join_table_field = "LINK_ID"
join_fields = ["ST_NAME"]
arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)

#on near table, join field from RCL NearFID = OID, add SEGMENTID, FULLNAME
print("joining SEGMENTID, FULLNAME from RCL...")
join_target = near
join_target_field = "NEAR_FID"
join_table = modal_composite
join_table_field = "OBJECTID"
join_fields = ["SEGMENTID","FULLNAME"]
arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)

#on near table, copy to new dataset keeping only rows with street name matches
print("dropping table rows with street name mismatches...")
near_select = os.path.join(scratch_gdb_path, "near_select")
arcpy.conversion.TableToTable(near, scratch_gdb_path,"near_select" , '"ST_NAME" = "FULLNAME"')

#on near select, run summary statistics, Case Field = SegmentID, SUM_Lanes = Median
print("generating summary stats table of SegmentID and median Lane count...")
lane_summary = os.path.join(scratch_gdb_path, "lane_summary")
arcpy.analysis.Statistics(near_select, lane_summary, statistics_fields = [["SUM_LANES", "MEDIAN"]], case_field = "SEGMENTID")

#on modal_composite, join field from near table on SegmentID, keeping lanes
print("joining final lane output to modal_composite...")
join_target = modal_composite
join_target_field = "SEGMENTID"
join_table = lane_summary
join_table_field = "SEGMENTID"
join_fields = ["MEDIAN_SUM_LANES"]
arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)

print("complete.")

### Lane Counts - parse & aggregate from multiple sources

In [ ]:
#fill lane_count with values from multiple fields. Will be used as main lane count indicator in analysis
#Unconstrained & Constrained Lane counts will be updated in subsquent functions/scripts

#1 - TransCAD
#2 - HPMS
#3 - 1986 ROW table
#4 - DTS data
#5 - Assumed Lanes by width where ROW < 100 and Class = A41

#fields to expose to update cursor
fields = ["SEGMENTID",       #0
          "FULLNAME",        #1
          "AB_LANE",         #2
          "BA_LANE",         #3
          "thrulanes",       #4
          "ln_exist_num",    #5
          "lane_count_exist",#6
          "lane_count_note", #7
          "mean_row_2020",   #8
          "ONEWAY",          #9
          "CLASS",           #10
          "MEDIAN_SUM_LANES",#11
          "lane_count_prop", #12
          "lane_count_rev"]  #13


count = 0
print("Aggregating lane count attributes...")
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if(row[2] is not None and row[3] is not None):
                lanes = row[2]+row[3]   
                row[6] = lanes
                row[12]= lanes
                row[13]= lanes
                
                row[7] = "Lanes values from TransCAD"
                count += 1  
                   
            elif row[4] is not None:
                
                lanes = row[4]
                row[6] = lanes
                row[12]= lanes
                row[13]= lanes
                row[7] = "Lanes values from HPMS"
                count += 1  
                
            elif(row[5] is not None):
                lanes = row[5]
                row[6] = lanes
                row[12]= lanes
                row[13]= lanes
                row[7] = "Lanes values from 1986 ROW Table"
                count += 1  
                
            elif(row[11] is not None):
                lanes = row[11]
                row[6] = lanes
                row[12]= lanes
                row[13]= lanes
                row[7] = "Lanes values from DTS data"
                count += 1  
                
            elif(row[8] < 100 and row[10] == "A41"):
                lanes = 2
                row[6] = lanes
                row[12]= lanes
                row[13]= lanes
                row[7] = "ROW <100, A41, assumed 2 lanes"
                count += 1  
                
            cursor.updateRow(row)
                        
        except (ValueError,TypeError) as error:
            
            print("Error at SegmentID: {0} - {1},  ".format(row[0],error))
            
print("Lane counts aggregated from {} items".format(count))


### Fill null values in ROW Estimate

In [ ]:
#Fill null ROW estimation values with assumption of 60' - revise if time/budget permitting

#fields to expose to update cursor
fields = ["SEGMENTID",
          "mean_row_2020",
          "row_estimate_note"]

where_clause = '"mean_row_2020" IS NULL'

count = 0
print("Filling ROW NULL values...")
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            row[1] = 60
            row[2] = "No Data. Assumed 60'."
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            
            print("Error at SegmentID: {0} - {1},  ".format(row[0],error))
            
print("ROW NULL values filled for {} items".format(count))

# 2. Modal Width Development

## 2.1 Calculate Modal Widths for Auto

Calculate current and unconstrained modal auto widths for streets throughout Oahu. Use lane counts and future hypothetical lane counts for congested street segments in combination with lane width parameter.

- Calculate existing modal with for auto traffic using lane count x average lane width into `modal_width_auto_exist`. 

- If VOC Max from TransCAD 2040 > 1.0, assume two-way streets have two additonal lanes added (22' additional width), and one-way streets have one additional lane added (11' additonal width). Proposed width is calcualted into `modal_width_auto_prop`

- If VOC Max from TransCAD 2040 is <1.0, `modal_width_auto_exist` is calculted into `modal_width_auto_prop`

**This section performs the following functions:**

1. Copy `lane_count_exist` into `lane_count_prop`.
2. Calculate current modal and unconstrained auto widths for segments where V/C >= 1.0. Add additonal lanes and required widths to unconstrained width and future lane count fields
3. Calculate current and unconstrained widths for segments where V/C < 1.0. (all values are equal)
4. Calcluate current and unconstrained widths for segmetns where V/C is NULL. (all values are equal)
5. Add 2' to all segments (current and unconstrained) where a median is present to account for additional spacing requirements. 


**Assumptions, Notes, and Parameters**
1. VOC Max values are from TransCAD 2040 dataset
2. Average lane width parameter =  11 
3. Auto Width calculated as (Lane Count x 11) + 2 (for addional spacing requirements on outtermost lanes)
4. To acount for median spacing requirments on innermost lane adjacet to medians, an additonal 2' is added for streets where medians are present
5. See Pre-Processing & Clean Up section for details on how lane counts are aggregated from input datasets

In [ ]:
#PARAMETERS
param_one_way = 75.0
param_two_way = 100.0

#fields to expose to update cursor
VOC_fields = ["SEGMENTID", #0
              "SEGMENTID", #1
              "SEGMENTID", #2
              "SEGMENTID", #3
              "SEGMENTID", #4
              "SEGMENTID", #5
              "SEGMENTID", #6
              "SEGMENTID", #7
              "SEGMENTID", #8
              "SEGMENTID", #9
              "ONEWAY",      #10
              "modal_width_auto_exist", #11
              "modal_width_auto_prop",  #12
              "AB_LANE",     #13
              "BA_LANE",     #14
              "SEGMENTID",   #15
              "ln_exist_num",#16
              "AB_VOC_MAX",  #17
              "BA_VOC_MAX",  #18 
              "mean_row_2020", #19
              "lanes_assumed", #20,
              "CLASS",         #21
              "VOC_max",    #22
              "thrulanes",   #23  HPMS Lanes
              "modal_width_auto_rule", #24,
              "lane_count_exist",   #25 - Lane Count - Existing
              "lane_count_prop"]   #26 - Lane Count - Proposed Unconstrained
             

print("Update rows for Traffic widths...")

count_01_01 = 0
count_01_02 = 0
voc_above = 0

count_02_01 = 0
count_03_01 = 0

count = 0
err_count = 0

#Set Future lanes to Currents Lanes - Future lanes will be updated by subsquent functions
print("set future lanes to current...")
where_clause = '"lane_count_exist" is NOT NULL'
with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            row[26] = row[25]
                     
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}".format(row[15],error))    

#CONDITIONAL GROUP 01 - VOC > 1.0
print("process VOC >= 1.0...")
where_clause = '("VOC_max" is NOT NULL) AND ("VOC_max" >= 1.0)'
with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #One Way Streets    
            if(
                (row[25] is not None) and    #Lanes
                (row[10] in [1,2])
            ):

                current_width = (row[25]*11) + 2
                expanded_width = current_width + 11              
                cond = "01_01" 
                row[11] = current_width
                row[12] = expanded_width
                row[26] = row[26] + 1     #add future lane
                row[24] = cond
                desc_exist = "{0} lanes * 11' per lane + 2' outside lane spacing = {1}' total".format(lanes, total)
                desc_prop = "{0} existing lanes + 1 future lane (V/C = {1} , One-Way Street) * 11' per lane + 2' outside lane spacing".format(lanes, total)
                voc_above +=1 
                count_01_01 +=1
                count +=1
                
            #Two Way Streets
            elif (
                (row[25] is not None) and  #Lanes
                (row[10] == 0)         #two-way indicator
            ):
                
                current_width = (row[25]*11) + 2
                expanded_width = current_width + 22    
                cond = "01_02"
                row[11] = current_width
                row[12] = expanded_width
                row[24] = cond
                row[26] = row[26] + 2     #add future lane
                voc_above +=1
                count_01_02 +=1
                count +=1
                     
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))        


#CONDITIONAL GROUP 02 - VOC < 1.0 - all records calclulated the same, no lanes added                
print("process VOC < 1.0...")    
where_clause = '("VOC_max" is NOT NULL) AND ("VOC_max" < 1.0)'
with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #All Streets
            if (row[25] is not None):
                                   
                current_width = (row[25]*11) + 2     
                cond = "02_01"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_02_01 +=1
                count +=1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))            
            

#CONDITIONAL GROUP 03 - VOC NULL              
print("process VOC IS NULL...")             
where_clause = '"VOC_max" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #All Streets
            if (row[25] is not None):
                                   
                current_width = (row[25]*11) + 2     
                cond = "03_01"
                row[11] = current_width
                row[12] = current_width
                row[24] = cond
                count_03_01 +=1
                count +=1
                                                                                    
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1}, {2} ".format(row[15],error,cond))
            

print("01-01 VOC >1.0 - One Way: {}".format(count_01_01))
print("01-02 VOC >1.0 - Two Way: {}".format(count_01_02))
print("VOC Above: {}".format(voc_above))

print("02-01 VOC <1.0 - Both Ways: {}".format(count_02_01))

print("03-01 VOC NULL - Both Ways:{}".format(count_03_01))



##Add 2' width to all auto widths on streets that contain medians (additional outside lane)
print("Add widths for roads w/ Medians...")
median_count = 0
where_clause = 'NOT ("median_type" IS NULL) AND NOT ("modal_width_auto_rule" IS NULL)'

with arcpy.da.UpdateCursor(modal_composite, VOC_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            if(row[11] is not None):    
                row[11] += 2
                row[12] += 2
                row[24] = row[24] + " MEDIAN"
                median_count += 1
        except ValueError as error:
            print("Error at SegmentID: {0} - {1}".format(row[15],error))
                     
print("median buffers calculated for {} items".format(median_count))

print("Modal Widths - Traffic complete.")

#### Results Confirmation

In [ ]:
#generate pandas dataframe frame
df = pd.DataFrame.spatial.from_featureclass(modal_composite)

In [ ]:
fields = ["SEGMENTID","FULLNAME","ONEWAY","VOC_max","modal_width_auto_rule","lane_count_exist","lane_count_prop","modal_width_auto_exist","modal_width_auto_prop"]

VOC = (df["VOC_max"].isnull())
VOC_above = (df["VOC_max"] >= 1.0)

lanes = (df["lane_count_exist"].notnull())

modal_rule = (df["modal_width_auto_rule"].isnull())

df.loc[VOC_above,fields]

In [ ]:
#QC Auto Conditionals
#Create DF for CSDM analysis

#01-01 - VOC >1.0 - One Way
filt_cond_1_1 = (~(df["lane_count_exist"].isnull()) &
         (df["VOC_max"] >1.0 ) &
         df["ONEWAY"].isin([1,2]))

#01-02 - VOC >1.0 - Two Way
filt_cond_1_2 = (~(df["lane_count_exist"].isnull()) &
         (df["VOC_max"] >1.0 ) &
         df["ONEWAY"].isin([0]))


#02-01 - VOC <1.0 - Both Ways
filt_cond_2_1 = (~(df["lane_count_exist"].isnull()) &
         (df["VOC_max"] <1.0 ))

#03-01 - VOC NULL - Both Ways
filt_cond_3_1 = (~(df["lane_count_exist"].isnull()) &
                 (df["VOC_max"].isnull()))

###
print("Conditional 01_01 check..")
print(df.loc[filt_cond_1_1,fields].shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='01_01'),fields].shape[0])
print("\n")

###
print("Conditional 01_02 check..")
print(df.loc[filt_cond_1_2,fields].shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='01_02'),fields].shape[0])
print("\n")

###
print("Conditional 02_01 check..")
print(df.loc[filt_cond_2_1,fields].shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='02_01'),fields].shape[0])
print("\n")

###
print("Conditional 03_01 check..")
print(df.loc[filt_cond_3_1,fields].shape[0])
print(df.loc[(df["modal_width_auto_rule"]=='03_01'),fields].shape[0])
print("\n")

## 2.2 Calculate Modal Widths for Bike Facilities

Calcluate current and unconstrained modal bike widths for street segments that contain bike facilities, or have proposed/redevelopment bike facilites

**This section performs the following functions:**

1. Calculate current and unconstrained widths for Bike Existing data
2. Calculate unconstrained widths for Bike Proposed data
3. Calculate unconstrained widths for Bike Redevelopment data


**Assumptions, Notes, and Parameters**
1. Bike Facility types do not differniate between one-way/two-way streets, or where bike facilites may be present on only one side of the street and not the other.
2. "Protected Bike Lane", "Buffered Bike Lane" = 12'
3. "Bike Lane", "Climbing Lane", "Shoulder Bikeway" = 7'


**Potential future improvements (not implemented) **
1. climbing lanes are only on one side = 7' regardles off one-way/two-way
2. bike lane and shoulder bike way should be doubled on 2 way street
3. buffered bike lane on one way street = 7'
4. protected bike lane = 14 - dont differentiate between one-way/two-way


In [ ]:
#fields to expose to update cursor
fields = ["Fac_Type_BE",            #0
          "modal_width_bike_exist", #1
          "Fac_Type_BP",            #2
          "Fac_Type_BR",            #3
          "modal_width_bike_prop"]  #4

count = 0
count_BE = 0
count_BR = 0
count_BP = 0

#update bike existing width columns
print("Update rows for Existing Bike facilities..")

where_clause = 'NOT ("Fac_Type_BE" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike existing calculations
            if (row[0] == "Protected Bike Lane" or
                row[0] == "Buffered Bike Lane"):
                row[1] = 12
                row[4] = 12
                    
            elif (row[0] == "Bike Lane" or
                  row[0] == "Climbing Lane" or
                  row[0] == "Shoulder Bikeway" or
                  row[0] == "Shared Use Path" or
                  row[0] == "Shared Roadway"):
                row[1] = 7
                row[4] = 7
                    
            cursor.updateRow(row)
            count += 1
            count_BE += 1
        except ValueError as error:
            print(error)            
print("done\n")

#Bike Proposed
#(will over write previous calcs if there is an improvement)
print("Update rows for Bike Proposed facilities..")

where_clause = 'NOT ("Fac_Type_BP" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite,fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #bike proposed & redev calculations
            if (row[2] == "Protected Bike Lane" or
                row[2] == "Buffered Bike Lane"):
                row[4] = 12
                
            elif (row[2] == "Bike Lane" or
                  row[2] == "Climbing Lane" or
                  row[2] == "Shoulder Bikeway" or
                  row[2] == "Shared Use Path" or
                  row[2] == "Shared Roadway"):
                row[4] = 7
                
            cursor.updateRow(row)
            count += 1
            count_BP += 1
        except ValueError as error:
            print(error)
print("done\n")
            
            
#Bike Redevlopment
#(will over write previous calcs if there is an improvement)
print("Update rows for Bike Re-development facilities..")

where_clause = 'NOT ("Fac_Type_BR" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite,fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            if (row[3] == "Protected Bike Lane" or
                  row[3] == "Buffered Bike Lane"):
                row[4] = 12
                
            elif (row[3] == "Bike Lane" or
                  row[3] == "Climbing Lane" or
                  row[3] == "Shoulder Bikeway" or
                  row[3] == "Shared Use Path" or
                  row[3] == "Shared Roadway"):
                row[4] = 7
                
            cursor.updateRow(row)
            count += 1
            count_BR += 1
        except ValueError as error:
            print(error)
print("done\n")
            
print("Bike Existing: {}".format(count_BE))
print("Bike Proposed: {}".format(count_BP))
print("Bike ReDev: {}".format(count_BR))
print("Total Bike records: {}".format(count))
    
print("\nModal Widths - Bike complete.")

In [ ]:
#Confirm Results
df = pd.DataFrame.spatial.from_featureclass(modal_composite)

fields = ["SEGMENTID",
          "Fac_Type_BE","modal_width_bike_exist",
          "Fac_Type_BP","modal_width_bike_prop",
          "Fac_Type_BR"]

BE = df["Fac_Type_BE"].notnull()
BE_w = df["modal_width_bike_exist"].notnull()
BP = df["Fac_Type_BP"].notnull()
BR = df["Fac_Type_BR"].notnull()
BP_w = df["modal_width_bike_prop"].notnull()

print("Bike Existing Records: {}".format(df.loc[BE,fields].shape[0]))
print("Bike Existing Widths: {}".format(df.loc[BE_w,fields].shape[0]))
print("\n")
print("Bike Proposed Records: {}".format(df.loc[BP,fields].shape[0]))
print("Bike Re-Dev Records: {}".format(df.loc[BR,fields].shape[0]))
print("Bike Proposed Widths: {}".format(df.loc[BP_w,fields].shape[0]))

## 2.3 Calculate Modal Widths for Medians

Median data was manually classified by reviewing street segments where `mean_row_2020 > 75`, then estimaing median type and median width using aerial imagery. Width estimates were then mapped into `modal_width_medians` (see assumptions below)

**This section performs the following functions:**

1. Calculate current and unconstrained widths for medians (all values the same)


**Assumptions, Notes, and Parameters**
1. Median width estimations were mapped to modal width values as follows:

|Median Width Estimate| Modal Width Value  |
|:--|:--|
| < 5' |5'  |
| 5' - 10' |11' (increase by 1' to match avg. lane width)  |
| 15' - 20' |17'  |
| >20' |22'  |


In [ ]:
#Calculate Modal Width for Medians

# "A","median_width"] = 5
# "B","median_width"] = 11
# "C","median_width"] = 17
# "D","median_width"] = 22

#fields to expose to update cursor
fields = ["median_type",               #0
          "median_width",              #1
          "modal_width_medians_exist", #2
          "modal_width_medians_prop"]  #3

#update median width columns
print("Update rows for Medians widths..")
count = 0
where_clause = 'NOT ("median_type" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite,fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        
        try:
            if row[1] == "A":
                median_width = 5
                row[2] = median_width
                row[3] = median_width
                count +=1
            elif row[1] == "B":
                median_width = 11
                row[2] = median_width
                row[3] = median_width
                count +=1
            elif row[1] == "C":
                median_width = 17
                row[2] = median_width
                row[3] = median_width
                count +=1
            elif row[1] == "D":
                median_width = 22
                row[2] = median_width
                row[3] = median_width
                count +=1
                    
            cursor.updateRow(row)
            
        except ValueError as error:
            print(error)
            
print("median widths calculated for {} items".format(count))
print("Modal Widths - Medians complete.")

In [ ]:
#Confirm Results
df = pd.DataFrame.spatial.from_featureclass(modal_composite)

df_fields = ["SEGMENTID",
          "median_type","median_width",
          "modal_width_medians_exist",
          "modal_width_medians_prop"]

medians = df["median_width"].notnull()
medians_ew = df["modal_width_medians_exist"].notnull()
medians_pw = df["modal_width_medians_prop"].notnull()

filt = df["median_width"] == "10-20"

print("Median Records: {}".format(df.loc[medians,df_fields].shape[0]))
print("Median Existing Widths: {}".format(df.loc[medians_ew,df_fields].shape[0]))
print("Median Proposed Widths: {}".format(df.loc[medians_pw,df_fields].shape[0]))

# df.loc[medians & ~medians_ew,fields]

## 2.4 Calculate Modal Widths for on-street Parking segments


Caclulate current and unconstrained modal widths for on-street parking, using data from DTS Sidewalk Inventory and assumptions about specific road types.


**This section performs the following functions:**

1. Calculate current and unconstrained widths using Road Buffer (NB)
2. Add calculated values for current and unconstrained widths using Rob Buffer (SB)
3. Calculate current and unconstrained width for segments not in Sidewalk Inventory but Streeet Class = A41
4. Calculate current and unconstrained widths to 0 for all other segments.


**Assumptions, Notes, and Parameters**
1. Average parking width parameter = 8
2. Please note that parking width is assumed througout the entire line segment. Does not factor in no parking zones, loading zones, etc.
3. Parking is not planned to be expanded in any current dataset. As such the "unconstrained" modal width for parking is identical to the current modal width for parking.

In [ ]:
#calculate modal width for street segments with on-street parking facilities

#expose fields to update cursor
parking_fields = ["SEGMENTID",              #0 - SegmentID
                  "NBRightP_7",             #1 - Road Buffer (NB)
                  "SBLeftPe_6",             #2 - Road Buffer (SB)
                  "NBRightP_4",             #3 - Ped Intrusion (NB)
                  "SBLeftPe_4",             #4 - Ped Intrusion (SB)
                  "modal_width_park_exist", #5 - Modal Width - Park (Existing)
                  "modal_width_park_prop",  #6 - Modal Width - Park (Proposed)
                  "mean_row_2020",          #7 - Mean ROW Analysis
                  "modal_width_auto_exist", #8 - Modal Width - Auto (Existing)
                  "CLASS"]                  #9 - Street Class (tiger)

park_ped_count = 0
park_assume_count = 0
no_can_park_count = 0
err_count = 0

param = 22

print("Update rows for Parking widths..")

#calculate parking width for segments based on ped inventory facilities for NB road side
where_clause = '"NBRightP_7" IS NOT NULL'
with arcpy.da.UpdateCursor(modal_composite, parking_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #Road Buffer(NB) or Ped Intrusion(NB)
            if((row[1] == 1) or (row[3] == 1)):               
                row[5] = 8
                row[6] = 8
                park_ped_count += 1
                              
            #NB Buffer & Intrusion = 0
            elif(row[1] == 0 and row[3] == 0):
                row[5] = 0
                row[6] = 0               
            cursor.updateRow(row)       
            
        except (ValueError,TypeError) as error:  
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

            
#calculate parking width for segments based on ped inventory facilities for SB road side
where_clause = '"SBLeftPe_6" IS NOT NULL' 
with arcpy.da.UpdateCursor(modal_composite, parking_fields, where_clause) as cursor:
    for row in cursor:
        
        try:
            #Road Buffer(SB) or Ped Intrusion(SB)
            if((row[2] == 1) or (row[4] == 1)):
                row[5] += 8
                row[6] += 8
                park_ped_count += 1
                
            #SB Buffer or Intrusion = 0 calc only required on rows skipped by previous cursor
            if((row[2] == 0 or row[4] == 0) and
                (row[6] is None)):
                row[6] = 0           
            cursor.updateRow(row)
            
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
               

#calculate parking width for segments not listed in ped inventory facilities for
#for A41 street class if ROW width - Auto Modal Width > parameter
where_clause = '"NBRightP_7" IS NULL AND "SBLeftPe_6" IS NULL AND "CLASS" = \'A41\''
with arcpy.da.UpdateCursor(modal_composite, parking_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[7] is not None) and
               (row[8] is not None) and
               (row[7] - row[8]) > param):
                
                row[5] = 16
                row[6] = 16
                park_assume_count +=1              
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
                      
            
#calculate parking width to 0 for all other segments, existing & proposed
where_clause = '"modal_width_park_exist" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, parking_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            row[5] = 0
            row[6] = 0
            no_can_park_count +=1               
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Parking widths (ped) calculated for {0} items".format(park_ped_count))
print("Parking widths (assumed) calculated for {0} items".format(park_assume_count))
print("Parking widths (0) calculated for {0} items".format(no_can_park_count))

print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Parking complete.")

## 2.5 Calculate Modal Widths for Ped Facilities


- Existing width is calculated into `modal_width_ped_exist` and `modal_width_ped_prop`

- Proposed width `modal_width_ped_prop` is then updated if a segment had no ped facilites, was not listed in ped facilites, or was listed in "Candidate Walkways" or "Candidate Upgrades", and the proposed width was greater than the existing width. (see assumptions below)

**This section performs the following functions:**
1. Calculate current and unconstrained widths to 0 for all segments where Ped Zone Width is NULL
2. Calculate current and unconstrained widths for all Ped Zone Width (NB) segments
3. Calculate and add current and unconstrained widths for all Ped Zone Width (SB) segments
4. Calculate and add current and unconstrained widths for all Pedestrian Buffer Furniture Zone (NB)
5. Calculate and add current and unconstrained widths for all Pedestrian Buffer Furniture Zone (SB)
6. Add **additonal dedicated space for Tier 2, 3, 4 Bus Stops** into unconstrained width in the pedestrian zone.
7. Increase unconstrained width for segemnts listed in Candidate Walkways and Candidate Upgrades.
8. Increase **unconstrained width to minimum of 6 for one-way streets and 12 for two-way streets**


**Assumptions, Notes, and Parameters**
1. Existing facilites from Pedestrian Sidewalk Inventory with data in "Pedestrian Zone Width" field is mapped as follows:
  
|Pedestrian Zone Width| Modal Width Value  |
|:--|:--|
| "Less than 5' " |4'  |
| "5' to 10' " |8'  |
| "Greater than 10' " |12'  |

2. Width values for "NB" and "SB" from Pedestrian Sidewalk Inventory are summed.
3. Any segment listed in `NewTier` == 4 is set to 8' if oneway, 16' if two-way for future widths.
3. The following steet segments types are set to 12' for `modal_width_ped_prop`
    1. Any street segment listed in "Candidate Walkways", and "Candidate Upgrades" datasets
    2. Any street segment listed in Pedestrian Sidewalk Inventory as having no facilites, but are not listed in "Candidate Walkways" or "Candidate Upgrades"
    3. Any street segment NOT listed in Pedestrian Sidewalk Inventory, AND not listed in "Candidate Walkways", AND not listed in "Candidate Upgrades", with `CLASS = A41`


### Setup

In [ ]:
#add field to hold furniture zone width

#store field, field alias in dict
new_fields = {
              "modal_width_ped_furn": "Modal Width - Ped (Furn)"
}

#loop through dict and add fields
for key,value in new_fields.items():
    print("Adding field {0}".format(key))
    arcpy.AddField_management(modal_composite,field_name=key,field_type="SHORT", field_alias = value)
    
print("Field additions complete")

In [ ]:
#calculate modal width for ped facilities

#assumptions based on ped data dictionary values for PED ZONE WIDTH
# greater than 10' = 12' DOMAIN 04
# 5'-10' (clear and obstruced types) = 8' DOMAIN 03, DOMAIN 02
# less than 5' = 4' DOMAIN 01

#Ped Zone Widths
p1 = 12 #DOMAIN VALUE 4          "Greater than 10' "
p2 = 8  #DOMAIN VALUE 3 and 2    "5' to 10' "
p3 = 4  #DOMAIN VALUE 1          "Less than 5' "

#existing ped facilities

#expose fields to update cursor
ped_fields = ["SEGMENTID",                #0 - SegmentID
                "NBRightP_1",             #1 - Pedestrian Zone Width (NB)
                "SBLeftPe_1",             #2 - Pedestiran Zone Width (SB)
                "modal_width_ped_exist",  #3 - Modal Width - Ped (Existing)
                "modal_width_ped_prop",   #4 - Modal Width - Ped (Proposed)
                "mean_row_2020",          #5 - Mean ROW Analysis
                "FID_Ped_Improve_temp",   #6 - Ped Improvements
                "FID_Ped_Add_temp",       #7 - Ped Additions
                "CLASS",                  #8 - Street Class (tiger)
                "NewTier",                #9 - Bus Transit - New Proposed Tier
                "ONEWAY",                 #10 - Oneway indicator
                "NBRightP_5",             #11 - Pedestrian Buffer Furniture Zone (NB)
                "SBLeftPedB",             #12 - Pedestrian Buffer Furniture Zone (SB)
                "Fac_Type_BE",            #13 - Bike - Existing Type
                "Fac_Type_BP",            #14 - Bike - Proposed Type
                "Fac_Type_BR",            #15 - Bike - ReDev Type
                "modal_width_park_exist", #16 - Modal Width - Park (Existing)
                "modal_width_park_prop",  #17 - Modal Width - Park (Proposed)
                "modal_width_ped_furn",   #18 - Modal Width - Ped (Furniture Zone)
                "modal_width_bus_prop"]   #19 - Modal Width - Bus (Existing)
     

In [ ]:
#Confirm Results
df = pd.DataFrame.spatial.from_featureclass(modal_composite)
df_fields = ["SEGMENTID",                #0 - SegmentID
                "NBRightP_1",             #1 - Pedestrian Zone Width (NB)
                "SBLeftPe_1",             #2 - Pedestiran Zone Width (SB)
                "modal_width_ped_exist",  #3 - Modal Width - Ped (Existing)
                "modal_width_ped_prop",   #4 - Modal Width - Ped (Proposed)
                "mean_row_2020",          #5 - Mean ROW Analysis
                "FID_Ped_Improve_temp",   #6 - Ped Improvements
                "FID_Ped_Add_temp",       #7 - Ped Additions
                "CLASS",                  #8 - Street Class (tiger)
                "NewTier",                #9 - Bus Transit - New Proposed Tier
                "ONEWAY",                 #10 - Oneway indicator
                "NBRightP_5",             #11 - Pedestrian Buffer Furniture Zone (NB)
                "SBLeftPedB",             #12 - Pedestrian Buffer Furniture Zone (SB)
                "Fac_Type_BE",            #13 - Bike - Existing Type
                "Fac_Type_BP",            #14 - Bike - Proposed Type
                "Fac_Type_BR",            #15 - Bike - ReDev Type
                "modal_width_park_exist", #16 - Modal Width - Park (Existing)
                "modal_width_park_prop",  #17 - Modal Width - Park (Proposed)
                "modal_width_ped_furn",   #18 - Modal Width - Ped (Furniture Zone)
                "modal_width_bus_prop"]   #19 - Modal Width - Bus (Existing)

In [ ]:
ped_NB = df["NBRightP_1"].notnull()
ped_SB = df["SBLeftPe_1"].notnull()

ped_EW = df["modal_width_ped_exist"].notnull()
ped_PW = df["modal_width_ped_prop"].notnull()

# print("Median Existing Widths: {}".format(df.loc[medians_ew,df_fields].shape[0]))
# print("Median Proposed Widths: {}".format(df.loc[medians_pw,df_fields].shape[0]))

# df.loc[ped_NB & ped_SB, ("modal_width_ped_exist","modal_width_ped_prop")].plot(figsize=(18,5), subplots = False)
df.loc[ped_NB & ped_SB, ("NBRightP_1","SBLeftPe_1","modal_width_ped_exist","modal_width_ped_prop")]

### Calculate widths from Sidewalk Inventory - Pedestrian Zone Width into Existing & Proposed

In [ ]:
count = 0
err_count = 0

print("Update rows with Ped Zone widths..")

print("Calculate widths for Null values")
where_clause = '("NBRightP_1" IS NULL) AND ("SBLeftPe_1" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields, where_clause) as cursor:
    for row in cursor:
        try:          
            row[3] = 0
            row[4] = 0
                 
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Calculate widths for NB Side...")
where_clause = 'NOT ("NBRightP_1" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields, where_clause) as cursor:
    for row in cursor:
        try:
            if row[1] == 4:
                width = 12
                row[3] = width
                row[4] = width       
            
            elif (row[1] in [2,3]):
                width = 8
                row[3] = width
                row[4] = width

            elif (row[1] == 1):
                width = 4
                row[3] = width
                row[4] = 4

            elif (row[1] == 0):
                width = 0
                row[3] = width
                row[4] = width
                 
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Calculate widths for SB Side")
where_clause = 'NOT ("SBLeftPe_1" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields, where_clause) as cursor:
    for row in cursor:
        try:
            if row[2] == 4:
                row[3] += 12
                row[4] += 12      
            
            elif (row[2] in [2,3]):
                row[3] += 8
                row[4] += 8

            elif (row[2] == 1):
                row[3] += 4
                row[4] += 4
                 
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
        


print("Done - processed {0} records, {1} errors".format(count, err_count))

### Calculate & Add Furniture Zone Widths to Existing & Proposed

In [ ]:
# Add furniture zone widths
print("Update rows by adding Ped Furniture widths..")

#Ped Furniture Widths
f1 = 6  #DOMAIN VALUE 3 - "Above 4' "
f2 = 4  #DOMAIN VALUE 2 - "0' to 4' "
f3 = 0  #DOMAIN VALUE 1 - "0 "

count = 0
err_count = 0

#Calc Nulls
print("Calculate Null values...")
where_clause = '("NBRightP_5" IS NULL) AND ("SBLeftPedB" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields, where_clause) as cursor:
    for row in cursor:
        try:          
            row[18] = 0
                 
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

#NB Records
print("Calculate NB Furn values...")
where_clause = 'NOT ("NBRightP_5" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
           
            # 6' - Domain 3
            if row[11] == 3:
                row[3] += 6
                row[4] += 6
                row[18] = 6
        
            # 4' - Domain 2
            elif row[11] == 2:             
                row[3] += 4
                row[4] += 4
                row[18] = 4
              
                   
           #0' fill for all others
            elif row[11] == 1:
                row[18] = 0
                 
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
#SB Records
print("Calculate SB Furn values...")
where_clause = 'NOT ("SBLeftPedB" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            
            # 6' - Domain 3
            if row[12] == 3:
                row[3] += 6
                row[4] += 6
                row[18] += 6
        
            # 4' - Domain 2
            elif row[12] == 2:             
                row[3] += 4
                row[4] += 4
                row[18] += 4
                           
            cursor.updateRow(row)
            count += 1
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
print("Done - processed {0} records, {1} errors".format(count, err_count))         

In [ ]:
#Confirm Results
df = pd.DataFrame.spatial.from_featureclass(modal_composite)
df_fields = ["SEGMENTID",                #0 - SegmentID
             "NBRightP_5",             #11 - Pedestrian Buffer Furniture Zone (NB)
             "SBLeftPedB",             #12 - Pedestrian Buffer Furniture Zone (SB)
             "modal_width_ped_furn"]   #18 - Modal Width - Ped (Furniture Zone)

In [ ]:
pedf_NB = df["NBRightP_5"].notnull()
pedf_SB = df["SBLeftPedB"].notnull()

pedf_EW = df["modal_width_ped_exist"].notnull()

# print("Median Existing Widths: {}".format(df.loc[medians_ew,df_fields].shape[0]))
# print("Median Proposed Widths: {}".format(df.loc[medians_pw,df_fields].shape[0]))

# df.loc[ped_NB & ped_SB, ("modal_width_ped_exist","modal_width_ped_prop")].plot(figsize=(18,5), subplots = False)
df.loc[pedf_NB, df_fields].plot.scatter(x = "NBRightP_5", y="modal_width_ped_furn",figsize=(18,8))

### Add additional dedicated widths for Bus Stops from Bus Transit NewTiers into Proposed

In [ ]:
print("Updating rows for NewTier values...")         
            
#Add additional required space for Bus Stops
count = 0
err_count = 0

where_clause = 'NOT ("NewTier" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields, where_clause) as cursor:
    for row in cursor:        
        try:
            #01 - Tier 3, 4 - Two Way Streets - if ped furn + parking < 20, add difference to ped
            if (
                (row[10] is not None) and
                (row[10] == 0) and         #OneWay
                (row[9] in [3,4]) and      #NewTier
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) < 20)    #Parking + Ped Furn
                ):
                
                cond = 1
                diff = (20-(row[16]+row[18]))
                row[4] = row[4]+diff
                
            #02 - Tier 2 - Two Way Strets + Protected Bike - if ped furn + parking < 20, add difference to ped   
            elif (
                (row[10] is not None) and
                (row[10] == 0) and           #OneWay
                (row[9] in [2]) and          #NewTier
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) < 20) and  #Parking + Ped Furn
                ((row[13] == "Protected Bike Lane") or (row[14] == "Protected Bike Lane") or (row[15] == "Protected Bike Lane"))
                ):
                            
                cond =2
                diff = (20-(row[16]+row[18]))
                row[4] = row[4]+diff
            
            #03 - Tier 3, 4 - One Way Streets - if ped furn + parking < 10, add difference to ped
            elif(
                (row[10] is not None) and
                (row[10] in [1,2]) and
                (row[9] in [3,4]) and
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) <10)
                ):
                
                cond=3
                diff = (10-(row[16]+row[18]))
                row[4] = row[4]+diff
                           
                           
            #04 - Tier 2 - One Way Streets + Protected Bike - if ped furn + parking < 10, add difference to ped   
            elif (
                (row[10] is not None) and
                (row[10] in [1,2]) and
                (row[9] in [2]) and
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) < 10) and
                ((row[13] == "Protected Bike Lane") or (row[14] == "Protected Bike Lane") or (row[15] == "Protected Bike Lane"))
                ):
                
                cond=4
                diff = (10-(row[16]+row[18]))
                row[4] = row[4]+diff
                           
            #05 - Tier 1 or (Tier 2 & Not Protected Bike) - Two Way Streets - if ped furn + parking < 16, add difference to ped
            elif (
                (row[10] is not None) and
                (row[10] in [0]) and
                (row[16] is not None) and
                (row[18] is not None) and
                (int((row[16]) + int(row[18])) < 16) and
                ((row[9] == 1) or (row[9] == 2 and not
                        ((row[13] == "Protected Bike Lane") or
                        (row[14] == "Protected Bike Lane") or
                        (row[15] == "Protected Bike Lane"))))
                ):
                
                cond=5
                sum_val = row[16]+row[18]
                diff = (16-(int(row[16])+int(row[18])))
                
                print("SegmentID: {0}, Parking: {1}, Furn: {2}, Total: {3}, Adding: {4}".format(row[0],row[16],row[18],sum_val, diff))
                row[4] = row[4] + int(diff)
            
            #06 - Tier 1 or (Tier 2 & Not Protected Bike) - One Way Streets - if ped furn + parking < 8, add difference to ped
            elif (
                (row[10] is not None) and
                (row[10] in [1,2]) and
                (row[16] is not None) and
                (row[18] is not None) and
                ((row[16] + row[18]) <8) and
                ((row[9] == 1) or (row[9] == 2 and not
                        ((row[13] == "Protected Bike Lane") or
                        (row[14] == "Protected Bike Lane") or
                        (row[15] == "Protected Bike Lane"))))
                ):
                
                cond = 6
                diff = (8-(row[16]+row[18]))
                row[4] = row[4]+diff
                                                 
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0}, Cond:{1}, Parking:{2}, Furniture:{3} - {4} ".format(row[0],cond,row[16], row[18], error))

### Ped Improvement & Candidate Walkways - increase future width to 12 if future width still below 12

In [ ]:
#proposed ped facilities - use a pass with update cursor to update proposed widths to 12' if they
#have segments listed in Ped improvement datasets, and the existing width is less than 12'
count = 0
err_count = 0

where_clause = 'NOT ("FID_Ped_Improve_temp" IS NULL OR "FID_Ped_Add_temp" IS NULL)'
with arcpy.da.UpdateCursor(modal_composite, ped_fields, where_clause) as cursor:
    for row in cursor:
        #print("row")
        try:
            #only update proposed width if is less than the current width
            if ((row[4] is not None) and row[4] < 12):
                row[4] = 12
                
            count += 1
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

print("done")

### 0' and No Data - set all  future widths to min of 12' if they are A41 street type

In [ ]:
# perform final pass with update cursor to add proposed width to facilites that have no sidewalks
# and are not covered by sidewalk inventory, but are residential (class = 'A41')
count = 0
err_count = 0

where_clause = '"NBRightP_1" IS NULL AND "FID_Ped_Improve_temp" IS NULL AND "FID_Ped_Add_temp" IS NULL'
with arcpy.da.UpdateCursor(modal_composite, ped_fields, where_clause) as cursor:
    for row in cursor:
        #print("final pass")
        try:
            #only update proposed width if is less than the current width
            if (row[8] == 'A41' and ((row[4] is not None and row[4] < 12) or row[4] is None)):
            #if ((row[8] == 'A41') and (row[4] is None)):
                row[4] = 12   
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            

print("Modal Widths - Ped complete.") 

## 2.6 Calculate Modal Widths for Bus Transit segments

Calculate current and unconstrained modal bus widths for street segments with Tier 4 bus service.

**This section performs the following functions:**
1. Calculuate existing modal bus widths for Tier 4 bus lanes.
2. Calculate unconstrained modal bus width for future Tier 4 bus lanes.

**Assumptions, Notes, and Parameters**
1. Existing Tier4 bus services routes were manually identifed by the project team and recorded with their SegmentID
2. This section only calcualtes modal widths for the **bus lane**. For widths allocated to **bus stops**, please see the previous section dedicated to pedestrian widths.

In [ ]:
#calculate modal width for bus facilities

#expose fields to update cursor
bus_fields = ["SEGMENTID",              #0 - SegmentID
              "NAME",                   #1 - Name
              "modal_width_bus_exist",  #2 - Modal Width - Bus (Existing)
              "modal_width_bus_prop",   #3 - Modal Width - Bus (Proposed)
              "mean_row_2020",          #4 - Mean ROW Analysis
              "CLASS",                  #5 - Street Class (tiger)
              "NewTier",                #6 - Bus Transit - New Tier
              "ONEWAY"]                 #7 - OneWay

bus_count = 0
new_count = 0
err_count = 0

#existing bus facilites by SegmentID (replace with Bus Plan data when available)
IDs_exist = (
    18813,18858,18867,18914,18968,18984,19024,19033,19068,19090,19140,19148,19148,19234,19272,
    19306,19320,19333,19357,19401,19441,19501,19595,19672,19758,19790,21993,22030,22049,22112,
    22163,22169,33146)

where_clause = '"SEGMENTID" IN {}'.format(IDs_exist)

print("Update rows for Bus widths..")
print("Calculating existing...")
#calculate existing Priority Bus facilities
with arcpy.da.UpdateCursor(modal_composite, bus_fields, '"SEGMENTID" IN {}'.format(IDs_exist)) as cursor:   
    for row in cursor:
        #print("row")
        try:
            if row[1] == "HOTEL":
                row[2] = 24
                row[3] = 24
                bus_count += 1
            
            elif row[1] == "KALAKAUA" or row[1] == "KING":
                row[2] = 12
                row[3] = 12
                bus_count += 1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))

print("Existing calc done.")
            
#calculate proposed Tier4 Bus facilities
print("Calculating proposed...")
with arcpy.da.UpdateCursor(modal_composite, bus_fields, '"NewTier" = 4') as cursor:
    for row in cursor:
        try:
            if ((row[7] is not None) and row[7] == 0):
                row[3] = 24
                new_count += 1
            elif((row[7] is not None) and row[7] in [1,2]):
                row[3] = 12
                new_count += 1
                
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
print("Proposed calc done.") 
            
print("Bus widths - Existing -  calculated for {0} segments".format(bus_count))
print("Bus widths - New -       calculated for {0} segments".format(new_count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Bus complete.")

# 3. Calculate Unconstrained Metrics

Use previously calculated modal widths for each modal type to generate a total unconstrained modal width value.


**Assumptions, Notes, and Parameters**
1. Future Tier 4 Bus Priority lanes are ignored in the unconstrained width calcuation based on the assumption that Bus Priority lanes will always be placed in existing auto lanes.
2. Do not sum Ped Width and Bike Width if future Bike Type = Shared Use Path
3. Ignore all bike facility types of shared road way in calculation



### Unconstrained Modal Width

In [ ]:
#calculate unconstrained modal widths

fields = ["SEGMENTID",               #0
          "modal_width_ped_prop",    #1
          "modal_width_bike_prop",   #2
          "modal_width_auto_prop",   #3
          "modal_width_bus_prop",    #4
          "modal_width_park_prop",   #5
          "modal_width_medians_prop",#6
          "modal_width_max",         #7
          "Fac_Type_BP",             #8
          "Fac_Type_BR",             #9
          "NewTier",                 #10
          "modal_width_max_desc"]    #11   

count = 0
err_count = 0

print("Calculating width..")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            #igonore shared road-way widths from bike width
            if ((row[8] == "Shared Roadway") or
                (row[9] == "Shared Roadway")):   
                 
                row[7] = sum(filter(None,(row[1], row[3], row[5], row[6])))
                row[11] = "Ped + Auto + Bus + Parking + Medians. Bike skipped due to Shared Roadway"
                count +=1
                
            #ignore bike shared use path for ped > 7    
            elif (((row[8] == "Shared Use Path") or
                  (row[9] == "Shared Use Path")) and
                  (row[1] is not None) and
                  (row[1] > 7)):
                 
                row[7] = sum(filter(None,(row[1], row[3], row[5], row[6])))
                row[11] = "Ped + Auto + Parking + Medians. Bike width skipped due to Shared Use Path & Proposed Ped > 7"
                count +=1
                
            #if shared use path is proposed on segment where future ped width is less than 7, assume 7 for ped width in calc   
            elif (((row[8] == "Shared Use Path") or
                  (row[9] == "Shared Use Path")) and
                  (row[1] is not None) and
                  (row[1] < 7)):
                 
                row[7] = sum(filter(None,(row[1], 7,  row[3], row[5], row[6])))
                row[11] = "Ped + 7' Bike Assumed + Auto + Bus + Parking + Medians. Shared Use Path on Ped < 7"
                count +=1
            
            #summarize all other segments - ignore Bus Lanes (assume area taken from auto, not widened)
            else:
                row[7] = sum(filter(None,(row[1], row[2], row[3], row[5], row[6])))
                row[11] = "Standard Summarization (Ped+Bike+Auto+Bus+Parking+Medians)"
                count +=1
                        
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
            
            
print("Unconstrained widths calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))
print("Modal Widths - Unconstrained complete.")


### Unconstrained Modal Areas

In [ ]:
#calculate additional modal metrics

fields = ["SEGMENTID",           #0
          "Shape_Length",        #1
          "length_ft",           #2
          "mean_row_2020",       #3
          
          "mean_row_area",       #4
          "modal_width_max",     #5
          "modal_width_max_diff",#6          
          "modal_area_max",      #7
          "modal_area_max_diff"] #8
        
count = 0
err_count = 0

print("Calculating areas..")
with arcpy.da.UpdateCursor(modal_composite, fields, 'NOT "mean_row_2020" IS NULL') as cursor:
    for row in cursor:
        #print("row")
        try:
            length_ft = row[2]
            mean_row_area = length_ft * row[3]
            modal_width_max_diff = row[5] - row[3]
            modal_area_max = length_ft * row[5]
            modal_area_diff = modal_area_max - mean_row_area
            
#             row[3] = length_ft
            row[4] = mean_row_area
            row[6] = modal_width_max_diff
            row[7] = modal_area_max
            row[8] = modal_area_diff
            
            count +=1
            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
                      
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))

print("Modal Areas - complete.")

In [ ]:
#calculate individual modal areas

fields = ["SEGMENTID",               #0
          "length_ft",               #1
          
          "modal_width_auto_exist",  #2
          "modal_width_auto_prop",   #3
          
          "modal_width_ped_exist",   #4
          "modal_width_ped_prop",   #5
          
          "modal_width_bike_exist", #6
          "modal_width_bike_prop",  #7
          
          "modal_area_auto_exist",#8
          "modal_area_auto_prop",#9
          
          "modal_area_ped_exist",#10
          "modal_area_ped_prop",#11
          
          "modal_area_bike_exist",#12
          "modal_area_bike_prop",#13
          
          "lane_count_exist", #14
          "lane_count_prop", #15
          "lane_miles_exist",       #16
          "lane_miles_prop",  #17
          
          "modal_width_park_exist", #18
          "modal_width_park_prop", #19
          "modal_area_park_exist", #20
          "modal_area_park_prop", #21
          
          "modal_width_medians_exist", #22
          "modal_width_medians_prop", #23
          "modal_area_medians_exist", #24
          "modal_area_medians_prop", #25

          "modal_width_bus_exist", #26
          "modal_width_bus_prop", #27
          "modal_area_bus_exist", #28
          "modal_area_bus_prop"] #29


print("Calculating modal areas..")



print("auto...")
count = 0
err_count = 0
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[2] is not None) and
               (row[3] is not None)):
                
                row[8] = row[2]*row[1] #Modal Area - Auto Existing
                row[9] = row[3]*row[1] #Modal Area - Auto Proposed (Unconstrained)
                count +=1
 
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))                        
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))


print("ped...")
count = 0
err_count = 0
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[4] is not None) and
               (row[5] is not None)):
            
                row[10] = row[4]*row[1] #Modal Area - Pedestrian Existing
                row[11] = row[5]*row[1] #Modal Area - Pedestrian Proposed (Unconstrained)
                count +=1
     
            cursor.updateRow(row)           
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))                       
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))



print("bike existing...")
count = 0
err_count = 0
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if(row[6] is not None):
                
                row[12] = row[6]*row[1] #Modal Area - Bike Existing
                row[13] = row[7]*row[1] #Modal Area - Bike Proposed (Unconstrained)
                count +=1
  
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))            
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))



print("bike proposed...")
count = 0
err_count = 0
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if(row[7] is not None):
                proposed_width = row[7]*row[1]
                
                row[13] = row[7]*row[1] #Modal Area - Bike Proposed (Unconstrained)
                count +=1
  
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))            
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))



print("parking...")
count = 0
err_count = 0
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[18] is not None) and
               (row[19] is not None)):
            
                row[20] = row[18]*row[1] #Modal Area - Pedestrian Existing
                row[21] = row[19]*row[1] #Modal Area - Pedestrian Proposed (Unconstrained)
                count +=1
     
            cursor.updateRow(row)           
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))                       
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))



print("medians...")
count = 0
err_count = 0
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[22] is not None) and
               (row[23] is not None)):
            
                row[24] = row[22]*row[1] #Modal Area - Medians Existing
                row[25] = row[23]*row[1] #Modal Area - Medians Proposed (Unconstrained)
                count +=1
     
            cursor.updateRow(row)           
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))                       
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))



print("bus lanes existing...")
count = 0
err_count = 0
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if(row[26] is not None):        
                row[28] = row[26]*row[1] #Modal Area - Bus Existing
                count +=1     
            cursor.updateRow(row)           
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))                       
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))


print("bus lanes proposed...")
count = 0
err_count = 0
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if(row[27] is not None):        
                row[29] = row[27]*row[1] #Modal Area - Bus Prop
                count +=1     
            cursor.updateRow(row)           
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))                       
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))



print("Lane miles...")
count = 0
err_count = 0
with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
            if((row[14] is not None) and
               (row[15] is not None)):
            
                row[16] = (row[14]*row[1])/5280   #Lane Miles (existing)
                row[17] = (row[15]*row[1])/5280
                count +=1
       
            cursor.updateRow(row)            
        except (ValueError,TypeError) as error:
            err_count += 1
            print("Error at SegmentID: {0} - {1} ".format(row[0],error))
print("Areas calculated for {0} items".format(count))
print("Errors (total) for {0} items".format(err_count))


print("Modal Areas 2 - complete.")

# 4. Results Analysis

In [3]:
#Confirm Results
df = pd.DataFrame.spatial.from_featureclass(modal_composite) 



###items below need cleanup

In [ ]:
fields_lanes = ["SEGMENTID",
                "length_ft",
                "lane_count_exist",
                "lane_count_note",
                "lane_count_prop",  
                "lane_miles_exist",
                "lane_miles_prop",
                "OWNER"]

fields_auto = ["SEGMENTID",
               "length_ft",
               "modal_width_auto_exist",
               "modal_width_auto_prop",
               "modal_area_auto_exist",
               "modal_area_auto_prop"]

fields_bike = ["SEGMENTID",
               "length_ft",
               "Fac_Type_BE",
               "modal_width_bike_exist",
               "Fac_Type_BP",
               "Fac_Type_BR",
               "modal_width_bike_prop",
               "modal_area_bike_exist",
               "modal_area_bike_prop"]

fields_parking = ["SEGMENTID",
                  "length_ft",
                  "modal_width_park_exist",
                  "modal_width_park_prop",
                  "modal_area_park_exist",
                  "modal_area_park_prop"]

fields_ped = ["SEGMENTID",
              "length_ft",
              "modal_width_ped_exist",
              "modal_width_ped_prop",
              "modal_area_ped_exist",
              "modal_area_ped_prop"]

fields_bus = ["SEGMENTID",
              "length_ft",
              "Tier",
              "NewTier",
              "modal_width_bike_exist",
              "modal_width_bike_prop",
              "modal_width_park_exist",
              "modal_width_park_prop",
              "modal_width_ped_exist",
              "modal_width_ped_prop"]

fields_metrics = ["SEGMENTID",
                  "OWNER",
                  "length_ft",
                  "mean_row_2020",
                  "mean_row_area",
                  "modal_width_max",
                  "modal_width_max_desc",
                  "modal_area_max",
                  "modal_area_diff_const"]

In [ ]:
auto = df["modal_area_auto_exist"].notnull()

bike_exist = df["modal_area_bike_exist"].notnull()
bike_prop = df["modal_area_bike_prop"].notnull()

ped_exist = df["modal_area_ped_exist"].notnull()
ped_prop = df["modal_area_ped_prop"].notnull()

lanes_exist = df["lane_count_exist"].notnull()
lanes_prop = df["lane_count_prop"].notnull()

lane_m_exist = df["lane_miles_exist"].notnull()
lane_m_prop = df["lane_miles_prop"].notnull()

parking = df["modal_area_park_exist"].notnull()

bus_exist = df["Tier"].notnull()
bus_new = df["NewTier"].notnull()

area = df["modal_area_max"].notnull()

length = df["length_ft"].notnull()

row = df["mean_row_2020"].notnull()

owner = (df["OWNER"] == "CITY") | (df["OWNER"] == "City") | (df["OWNER"] == "VARIOUS")

above = (df["modal_width_max"] > df["mean_row_2020"])

below = (df["modal_width_max"] < df["mean_row_2020"])

total_miles = round(df.loc[length, fields_metrics].sum()[2]/5280,0)
project_miles = round(df.loc[length & owner, fields_metrics].sum()[2]/5280,0)

total_area = round(df.loc[length, fields_metrics].sum()[4]/43560,0)
project_area = round(df.loc[length & owner, fields_metrics].sum()[4]/43560,0)

auto_miles_exist = round(df.loc[auto & owner, fields_auto].sum()[1]/5280,0)
auto_area_exist = round(df.loc[auto & owner, fields_auto].sum()[4]/43560,0)
auto_area_prop = round(df.loc[auto & owner, fields_auto].sum()[5]/43560,0)

lane_miles_exist = round(df.loc[length & owner & lane_m_exist, fields_lanes].sum()[5],1)
lane_miles_prop = round(df.loc[length & owner & lane_m_exist, fields_lanes].sum()[6],1)

bike_miles_exist = round(df.loc[bike_exist & owner, fields_bike].sum()[1]/5280,0)
bike_miles_prop = round(df.loc[bike_prop & owner, fields_bike].sum()[1]/5280,0)
bike_area_exist = round(df.loc[bike_exist & owner, fields_bike].sum()[6]/43560,0)
bike_area_prop = round(df.loc[bike_prop & owner, fields_bike].sum()[7]/43560,0)

park_area_exist = round(df.loc[parking & owner, fields_parking].sum()[4]/43560,0)

ped_miles_exist = round(df.loc[ped_exist & owner, fields_ped].sum()[1]/5280,0)
ped_miles_prop = round(df.loc[ped_prop & owner, fields_ped].sum()[1]/5280,0)
ped_area_exist = round(df.loc[ped_exist & owner, fields_ped].sum()[4]/43560,0)
ped_area_prop = round(df.loc[ped_prop & owner, fields_ped].sum()[5]/43560,0)

print("Total Miles (all data): {}".format(total_miles))
print("Total Miles (City & Various): {}".format(project_miles))
print("Total Acres (all data): {}".format(total_area))
print("Total Acres (City & Various): {}".format(project_area))
print("\n")
print("Unconstrained Width > ROW (miles): {}".format(df.loc[above & owner,fields_metrics].sum()[2]/5280))
print("Unconstrained Width < ROW (miles): {}".format(df.loc[below & owner,fields_metrics].sum()[2]/5280))
print("\n")
print("###Auto###")
print("Miles Existing: {}".format(auto_miles_exist))
print("Miles Proposed: N/A")
print("Acres Existing: {}".format(auto_area_exist))
print("Acres Proposed: {}".format(auto_area_prop))
print("\n")
print("###Transit Lanes###")
print("Lane Miles Existing: {}".format(lane_miles_exist))
print("Lane Miles Proposed: {}".format(lane_miles_prop))
print("\n")
print("###Bike###")
print("Miles Existing: {}".format(bike_miles_exist))
print("Miles Proposed: {}".format(bike_miles_prop))
print("Acres Existing: {}".format(bike_area_exist))
print("Acres Proposed: {}".format(bike_area_prop))
print("\n")
print("###Parking###")
print("Acres Existing: {}".format(park_area_exist))
print("\n")
print("###Ped###")
print("Miles Existing: {}".format(ped_miles_exist))
print("Miles Proposed: {}".format(ped_miles_prop))
print("Acres Existing: {}".format(ped_area_exist))
print("Acres Proposed: {}".format(ped_area_prop))


# 5. Reset Data Functions

In [ ]:
#reset all (or some) width and area fields to null (use during development)

fields = [
          
          "modal_width_auto_exist",   #0
          "modal_width_auto_prop",    #1
          "modal_area_auto_exist",    #2
          "modal_area_auto_prop",     #3
    
          "lane_count_prop",          #4
          "lane_miles_exist",         #5
          "lane_miles_prop",          #6
    
          "modal_width_bike_exist",   #7
          "modal_width_bike_prop",    #8
          "modal_area_bike_exist",    #9
          "modal_area_bike_prop",     #10
    
          "modal_width_medians_exist",#11
          "modal_width_medians_prop", #12
          "modal_area_medians_exist", #13
          "modal_area_medians_prop",  #14
    
          "modal_width_park_exist",   #15
          "modal_width_park_prop",    #16  
          "modal_area_park_exist",    #17
          "modal_area_park_prop",     #18

          "modal_width_ped_exist",    #19
          "modal_width_ped_prop",     #20  
          "modal_area_ped_exist",     #21
          "modal_area_ped_prop",      #22

          "modal_width_bus_exist",    #23
          "modal_width_bus_prop",     #24   
          "modal_area_bus_exist",     #25
          "modal_area_bus_prop",      #26

          "mean_row_area",            #27
          "modal_width_exist",        #28
          "modal_width_max",          #29
          "modal_width_max_diff",     #30
          "modal_width_max_desc",     #31
  
          "modal_area_max",           #32
          "modal_area_max_diff"]      #33


err_count = 0

print("Reseting modal widths and areas to Null...")

with arcpy.da.UpdateCursor(modal_composite, fields) as cursor:
    for row in cursor:
        #print("row")
        try:
                                 
#             row[0] = None    # modal_width_auto_exist
#             row[1] = None    # modal_width_auto_prop
#             row[2] = None    # modal_area_auto_exist
#             row[3] = None    # modal_area_auto_prop
            
#             row[4] = None    # lane_count_prop
#             row[5] = None    # lane_miles_exist
#             row[6] = None    # lane_miles_prop
            
#             row[7] = None    # modal_width_bike_exist
#             row[8] = None    # modal_width_bike_prop
#             row[9] = None    # modal_area_bike_exist
#             row[10] = None   # modal_area_bike_prop
            
#             row[11] = None   # modal_width_medians_exist
#             row[12] = None   # modal_width_medians_prop
#             row[13] = None   # modal_area_medians_exist
#             row[14] = None   # modal_area_medians__prop
            
#             row[15] = None   # modal_width_park_exist
#             row[16] = None   # modal_width_park_prop
#             row[17] = None   # modal_area_park_exist
#             row[18] = None   # modal_area_park_prop
            
            row[19] = None    # modal_width_ped_exist
            row[20] = None    # modal_width_ped_prop
            row[21] = None    # modal_area_ped_exist
            row[22] = None    # modal_area_ped_prop
            
#             row[23] = None   # modal_width_bus_exist
#             row[24] = None   # modal_width_bus_prop
#             row[25] = None   # modal_area_bus_exist
#             row[26] = None    # modal_area_bus_prop
            
            row[27] = None    # mean_row_area
            row[28] = None    # modal_width_exist
            row[29] = None    # modal_width_max
            row[30] = None    # modal_width_max_diff
            row[31] = None    # modal_width_max_desc
            
            row[32] = None    # modal_area_max
            row[33] = None    # modal_area_max_diff



            cursor.updateRow(row)
            
        except (ValueError,TypeError) as error:
            err_count += 1
            print(error)

print("All widths and areas reset to Null.")
            